In [ ]:
from pathlib import Path
import tarfile

from pybuild_deps.find_build_dependencies import find_build_dependencies
from pybuild_deps.get_package_source import get_package_source

from cachi2.core.package_managers.pip import PipRequirementsFile

LOCK_FILE = "Cargo.lock"
PACKAGE_FILE = "Cargo.toml"

def depends_on_rust(deps: list) -> bool:
    for d in deps:
        if d.startswith("setuptools-rust"):
            return True
    return False

def _save_dep_file(tarball, full_file_name, path_to_dependencies_file):
    file_handle = tarball.extractfile(full_file_name)
    path_to_dependencies_file.parent.mkdir(exist_ok=True, parents=True)
    path_to_dependencies_file.write_bytes(file_handle.read())

def find_rust_deps(requirements_file):
    req_file = PipRequirementsFile(Path(requirements_file))
    for req in req_file.requirements:
        package_name = req.package
        version_qualifier, version = req.version_specs[0]
        assert version_qualifier == "==", "an EXACT match is required."
        build_dependencies = find_build_dependencies(package_name, version)
        if not depends_on_rust(build_dependencies):
            continue
        source_code = get_package_source(package_name, version)
        with tarfile.open(fileobj=source_code.open("rb")) as tarball:
            for full_file_name in tarball.getnames():
                # this is assuming a source code contains only one cargo.toml/lock, 
                # which might not be true - (for the packages used in this exercise it 
                # is OK)
                if full_file_name.endswith((LOCK_FILE, PACKAGE_FILE)):
                    file_name = Path(full_file_name).name
                    path_to_dependencies_file = (
                        req_file.file_path.parent / "dependencies" / f"{package_name}-rust" / file_name
                    )
                    _save_dep_file(tarball, full_file_name, path_to_dependencies_file)

In [ ]:
!git clone git@github.com:quipucords/quipucords.git --depth 1 --branch=1.2.4

In [ ]:
find_rust_deps("quipucords/requirements.txt")
find_rust_deps("quipucords/requirements-build.txt")

In [ ]:
!ls quipucords/dependencies

In [47]:
!cachi2 fetch-deps --source quipucords '[{"type": "pip"},{"type": "cargo", "path": "dependencies/cryptography-rust"},{"type": "cargo", "path": "dependencies/bcrypt-rust"}]'

2023-04-28 10:35:54,211 INFO Extracting metadata from setup.py
2023-04-28 10:35:54,214 INFO Found name in setup.py: 'quipucords'
2023-04-28 10:35:54,215 INFO Found version in setup.py: '0.0.0'
2023-04-28 10:35:54,215 INFO Resolved package name: 'quipucords'
2023-04-28 10:35:54,215 INFO Resolved package version: '0.0.0'
2023-04-28 10:35:54,248 INFO No hash options used, will not require hashes unless HTTP(S) dependencies are present.
2023-04-28 10:35:54,249 INFO Downloading amqp==5.1.1 ; python_version >= "3.9" and python_version < "4.0"
2023-04-28 10:35:54,467 INFO Successfully downloaded amqp==5.1.1 ; python_version >= "3.9" and python_version < "4.0" to deps/pip/amqp-5.1.1.tar.gz
2023-04-28 10:35:54,467 INFO Downloading ansible-core==2.13.8 ; python_version >= "3.9" and python_version < "4.0"
2023-04-28 10:35:55,223 INFO Successfully downloaded ansible-core==2.13.8 ; python_version >= "3.9" and python_version < "4.0" to deps/pip/ansible-core-2.13.8.tar.gz
2023-04-28 10:35:55,223 INFO

In [38]:
%%writefile cargo_config.toml
[source.crates-io]
replace-with = "local"

[source.local]
local-registry = "/tmp/cachi2-output"


Overwriting cargo_config.toml


In [ ]:
%%writefile Containerfile.baseimage
FROM registry.redhat.io/ubi8/ubi

ARG BUILDREQUIRES='cargo gcc libffi-devel libpq-devel libtool openssl-devel patch python39-devel rust rust-std-static llvm-libs'
RUN dnf install \
        git \
        glibc-langpack-en \
        jq \
        libpq \
        make \
        openssh-clients \
        python39 \
        sshpass \
        tar \
        which \
        $BUILDREQUIRES \
        -y &&\
    dnf clean all

In [ ]:
!podman build --tag quipucords-base-image -f Containerfile.baseimage .

In [48]:
# generating cargo indexes should be done only for the downloaded packages - 
# but for now lets skip that (I still don't know how to do it XD) and just clone the 
# whole thing
!git clone git@github.com:rust-lang/crates.io-index.git --depth 1 cachi2-output/index

Cloning into 'cachi2-output/index'...
remote: Enumerating objects: 137832, done.
remote: Counting objects: 100% (137832/137832), done.
remote: Compressing objects: 100% (55322/55322), done.
remote: Total 137832 (delta 74907), reused 127947 (delta 68857), pack-reused 0
Receiving objects: 100% (137832/137832), 71.75 MiB | 10.63 MiB/s, done.
Resolving deltas: 100% (74907/74907), done.
Updating files: 100% (112612/112612), done.


In [50]:
!cachi2 inject-files "$(realpath ./cachi2-output)" --for-output-dir /tmp/cachi2-output

In [51]:
!cachi2 generate-env "$(realpath ./cachi2-output)" -o $(pwd)/cachi2.env --for-output-dir /tmp/cachi2-output

quipucords also has a UI. let's build it separately and ignore cachi2 for the sake 
of simplicity

In [ ]:
!git clone git@github.com:quipucords/quipucords-ui.git --depth 1 --branch=1.2.0

In [30]:
%%writefile Containerfile.quipucords-ui
FROM registry.access.redhat.com/ubi8/nodejs-16

USER root
RUN npm --userconfig .npmrc install --global yarn
# Install/build UI
COPY quipucords-ui /app
WORKDIR /app

RUN yarn install &&\
    yarn build:brand

Overwriting Containerfile.quipucords-ui


In [31]:
!podman build --tag quipucords-ui -f Containerfile.quipucords-ui .

STEP 1/6: FROM registry.access.redhat.com/ubi8/nodejs-16
STEP 2/6: USER root
--> Using cache 2b5a30209585694c84a5eef77d9050d16c96e9cbd99cd3ccd5c5175855ed0fe4
--> 2b5a30209585
STEP 3/6: RUN npm --userconfig .npmrc install --global yarn

added 1 package, and audited 2 packages in 896ms

found 0 vulnerabilities
npm notice 
npm notice New major version of npm available! 8.19.3 -> 9.6.5
npm notice Changelog: <https://github.com/npm/cli/releases/tag/v9.6.5>
npm notice Run `npm install -g npm@9.6.5` to update!
npm notice 
--> e3bdf0ef69c3
STEP 4/6: COPY quipucords-ui /app
--> 6b16287576ac
STEP 5/6: WORKDIR /app
--> 6ead0bcc9dd6
STEP 6/6: RUN yarn install &&    yarn build:brand
yarn install v1.22.19
[1/5] Validating package.json...
[2/5] Resolving packages...
[3/5] Fetching packages...
[4/5] Linking dependencies...
warning " > bootstrap@5.1.3" has unmet peer dependency "@popperjs/core@^2.10.2".
warning " > patternfly-react@2.40.0" has incorrect peer dependency "react@^16.3.2".
warning " > patt

In [64]:
%%writefile Containerfile

# FROM quipucords-ui:latest as yarn_builder

FROM quipucords-base-image:latest

# setup python virtualenv
RUN python3 -m venv /opt/venv
ENV PATH="/opt/venv/bin:${PATH}"

WORKDIR /app
# copy cargo config (this should become part of cargo inject IF crates are available 
# from local files - only problem is this file needs to be at an specific location)
# NOTICE the config is placed on root directory - this was required to successfully
# build indirect rust dependencies (cryptography and bcrypt). Looks like pip is running
# not running the build under /app
COPY cargo_config.toml /.cargo/config.toml
COPY quipucords/requirements.txt .


RUN source /tmp/cachi2.env && \
    pip install -U pip 'setuptools<67' wheel &&\
    # build/install kombu first, as it requires an older setuptools and dont follow PEP-517
    pip install kombu &&\
    pip install -U setuptools &&\
    pip install -r requirements.txt

# copy quipucords source
COPY quipucords /app
# copy UI code
COPY --from=quipucords-ui:latest /app/dist/client quipucords/client
COPY --from=quipucords-ui:latest /app/dist/templates quipucords/quipucords/templates

ENV DJANGO_DEBUG=False
ENV LANG=C
ENV LC_ALL=C
ENV PRODUCTION=True
ENV PYTHONPATH=/app/quipucords

RUN make server-static

EXPOSE 443
COPY quipucords/deploy /deploy
COPY quipucords/deploy/ssl/ /etc/ssl/qpc/

CMD ["/bin/bash", "/deploy/docker_run.sh"]


Overwriting Containerfile


In [65]:
!podman build . \
  --volume "$(realpath ./cachi2-output)":/tmp/cachi2-output:Z \
  --volume "$(realpath ./cachi2.env)":/tmp/cachi2.env:Z \
  --network none \
  --tag quipucords-no-network

STEP 1/20: FROM quipucords-base-image:latest
STEP 2/20: RUN python3 -m venv /opt/venv
--> Using cache d9185e6f5c2fdfd47fe6c943907b170ce6b3b03ace40d93e43a64b1e3b67e728
--> d9185e6f5c2f
STEP 3/20: ENV PATH="/opt/venv/bin:${PATH}"
--> Using cache d67fe7085a7c7ece58f8601fe1eed6107fdc2d5b0cc6c1399aaf88696b4dc267
--> d67fe7085a7c
STEP 4/20: WORKDIR /app
--> Using cache 021917a443648249d499c6fac3f0fd3d3829a519026694e921dc1c12f8ce0289
--> 021917a44364
STEP 5/20: COPY cargo_config.toml /.cargo/config.toml
--> Using cache 00d1a1cd976da0dfecdbe8202ed4fe6f3ebd5ec259cc048701b41f95db2ab56d
--> 00d1a1cd976d
STEP 6/20: COPY quipucords/requirements.txt .
--> Using cache 08e1ba082e883e1f3599048312811dcba882c35b8dc52e30259ec4497a274363
--> 08e1ba082e88
STEP 7/20: RUN source /tmp/cachi2.env &&     pip install -U pip 'setuptools<67' wheel &&    pip install kombu &&    pip install -U setuptools &&    pip install -r requirements.txt
--> Using cache ffcac5df9f0544f7a22db8b6608358aba746c42df6c43840fc1ad6936f5c